This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
pip install Quandl

You should consider upgrading via the '/Users/carolineliu/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import quandl
import json

In [4]:
quandl.ApiConfig.api_key = API_KEY

In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
glimpse = quandl.get('FSE/AFX_X', start_date='2017-01-01', end_date='2017-01-02', returns='json')

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
glimpse

,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
Date,,,,,,,,,,
2017-01-02,34.99,35.94,34.99,35.8,None,44700.0,1590561.0,None,None,None


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Q1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
(keep in mind that the date format is YYYY-MM-DD).

In [7]:
import requests
import numpy as np

In [8]:
url =  "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key="
r = requests.get(url)
json_data = r.json()


In [9]:
dict_data=json_data['dataset']

In [10]:
dict_key = dict_data['column_names']

In [11]:
dict_values = dict_data['data']

In [12]:
len(dict_values)

255

# Q2. Convert the returned JSON object into a Python dictionary.

In [13]:
AFX_dict = {}

In [17]:
for i in range(len(dict_key)):
    for j in range(len(dict_values)):
        if dict_key[i] in AFX_dict:
            list(AFX_dict[dict_key[i]])
            AFX_dict[dict_key[i]].append(dict_values[j][i])
        else:
            AFX_dict[dict_key[i]] = dict_values[j][i]

# Q3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [19]:
open_price = AFX_dict['Open'][1:].copy()

In [20]:
open_price = [x for x in open_price if x is not None]

In [21]:
max(open_price), min(open_price)

(53.11, 34.0)

In [22]:
for i in range(len(dict_values)):
    if dict_values[i][1] == max(open_price):
        print('Q3 answer:')
        print(dict_values[i][0]+"'s opening price is {}, it's the highest opening price for the stock in this period.".format(max(open_price)))
    elif dict_values[i][1] == min(open_price):
        print(dict_values[i][0]+"'s opening price is {}, it's the lowest opening price for the stock in this period.".format(min(open_price)))


Q3 answer:
2017-12-14's opening price is 53.11, it's the highest opening price for the stock in this period.
2017-01-24's opening price is 34.0, it's the lowest opening price for the stock in this period.


# Q4. What was the largest change in any one day (based on High and Low price)?

In [23]:
High = AFX_dict['High'][1:].copy()
Low = AFX_dict['Low'][1:].copy()

In [24]:
change = []
for i in range(len(High)):
    change.append(High[i]-Low[i])

In [25]:
max(change)

2.8100000000000023

In [26]:
for i in range(len(dict_values)):
    if dict_values[i][2] - dict_values[i][3] == max(change):
        print("Q4 answer: The stock has the largest change of the price ${:.2f} on {}".format(max(change),dict_values[i][0]))

Q4 answer: The stock has the largest change of the price $2.81 on 2017-05-11


# Q5. What was the largest change between any two days (based on Closing Price)?

In [27]:
Close = AFX_dict['Close'][1:].copy()

In [28]:
daily_change = []
for i in range(len(Close)):
    daily_change.append(Close[i] - Close[i-1])

In [29]:
daily_change=daily_change[1:]

In [30]:
for i in range(len(dict_values)):
    if dict_values[i][4] - dict_values[i-1][4] == max(daily_change):
        print('Q5 answer: The change of closing price between {} and {} is the largest, ${:.2f}'.format(dict_values[i][0], dict_values[i-1][0], max(daily_change)))

Q5 answer: The change of closing price between 2017-08-08 and 2017-08-09 is the largest, $2.56


# Q6. What was the average daily trading volume during this year?

In [31]:
dict_key

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [32]:
volume = AFX_dict['Traded Volume'][1:].copy()

In [33]:
avg_volume = sum(volume) / len(volume)
print('Q6 answer: The average daily trading volume during 2017 is ${:.2f}'.format(avg_volume))

Q6 answer: The average daily trading volume during 2017 is $89124.34


# Q7. What was the median trading volume during this year. 
(Note: you may need to implement your own function for calculating the median.)

In [34]:
def median(data):
    data.sort()
    if len(data)%2 == 0:
        i = int(len(data)/2)
        median = (data[i] + data[i+1]) / 2
    else:
        i = int((len(data)+1)/2)
        median = data[i]
    return median
    

In [36]:
med_vol = median(volume)
print('Q7 answer: The median trading volume during 2017 is ${:.2f}'.format(med_vol))

Q7 answer: The median trading volume during 2017 is $76600.00
